# RBX
## Övergripande arbetsgång
    * Importerar punkter från DGN-fil i arbetsmapp till den aktuella databasen.
      Hämtar in punkter till databasen och placerar dem under /RBX_clean
    * Beräknar avstånd från importerade punkter till fasledningar.
      Fasledningar har tidigare hämtats in från Wires.ipynb
    * I RBX_polygon.py skapas polygoner, grupperar punkter och hämtar de punkter som har minst avstånd till fas.
      Polygoner och punkter skrivs ut till shapefiler (.shp)
    * Polygoner och punkter hämtas in till geodatabasen, från shapefilerna, och lägger även till domäner, ledningsgata(nr)
      och littera.

## Sökvägar

In [ ]:
# Läs settings
import json
import os
import arcpy
import pandas as pd
import geopandas as gpd
import pyproj

settings_file = r"Q:\Projekt\Data_2024\styrfiler\settings_SEKNNO_postgis.json"
#settings_file = r"Q:\Projekt\Data_2024\styrfiler\settings_SEVPLI.json"
#settings_file = r"C:\SVK_utveckling\settings_SEVPLI.json"


with open(settings_file, 'r', encoding='utf-8') as file:
    settings = json.load(file)

#print(settings)
print(json.dumps(settings, indent=4))

run_ID = settings["run_ID"]
powerline_list = settings["powerline_list"]
local_dir = settings["local_folder"]
powerlines_folder = settings["powerlines_folder"]
domains_folder = settings["domains_folder"]
dgn_dir = settings["dgn_folder"]
scandate_file = settings["scandate_file"]
working_gdb_template = settings["working_gdb_template"]
results_gdb_template = settings["results_gdb_template"]
cvd_LEDNINGSGATA_path = os.path.join(domains_folder, "cvd_LEDNINGSGATA.txt")
wires_gdb = os.path.join(local_dir, f"wires_{run_ID}.gdb")
RBX_shape_folder = os.path.join(local_dir, f"RBX_shp_{run_ID}")
working_gdb = os.path.join(local_dir, f"working_{run_ID}.gdb")
results_gdb = os.path.join(local_dir, f"results_{run_ID}_PostGIS.gdb")
module_path = settings["modules"]
proj_lib_path = settings["proj_lib_path"]

pyproj.datadir.set_data_dir(proj_lib_path)

print("körning av cell klar")


# Import modules

In [ ]:
#Load rbx_utils.py
import sys
import importlib

# Den absoluta sökvägen till modules
print(module_path)  # Kontrollera att sökvägen är korrekt
if module_path not in sys.path:
    sys.path.append(module_path)

# Importera rbx_utils
import rbx_utils
importlib.reload(rbx_utils)

# test om utils är laddat
rbx_utils.is_rbx_utils_loaded()


# Skapa lokala kataloger och kopiera template-gdb:er

In [ ]:
from pathlib import Path
import os
import shutil

# Create empty folder for shapefiles
if os.path.exists(RBX_shape_folder):
    print(f"{RBX_shape_folder} finns redan")
else:
    os.makedirs(RBX_shape_folder)

# Create working gdb as copy of template
if os.path.exists(working_gdb):
    print(f"{working_gdb} finns redan")
else: 
    shutil.copytree(working_gdb_template, working_gdb)

# Create results gdb as copy of template
if os.path.exists(results_gdb):
    print(f"{results_gdb} finns redan")
else: 
    shutil.copytree(results_gdb_template, results_gdb)
print("körning av cell klar")

# Slår ihop RBX-filer för blocken till en enda fil för ledningen
### Behöver inte köras, den som gör ledningen i TerraScan gör detta själv

### skapa kontroll om rbx finns eller inte, om inte finns varna och gör ihopslagning

In [ ]:
#import os
#import sys
#import glob
#import fileinput
#import SVK
#import pandas as pd

In [ ]:
## Merge RBX txt files for all blocks of a powerline (LGXXX_Y) into one txt
#import fileinput
#
#def merge_blocks(src_dir, search_pattern, dst_file):
#    blocks = glob.glob(os.path.join(src_dir, search_pattern))
#    with open(dst_file, "w") as fh:
#        input_lines = fileinput.input(blocks)
#        fh.writelines(input_lines)
#        
#def combine_blocks(row):
#    LG = row["LG"]
#    line = row["line"]
#    line_dir = serv_prj_dir / LG / f"line_{line}"
#    
#    print(f"Doing {LG}_{line}")
#    block_dir = os.path.join(line_dir, "RBX", "block")
#    combined_blocks_path = os.path.join(line_dir, "RBX", "RBX_raw.txt")
#    merge_blocks(block_dir, "*_pt*.txt", combined_blocks_path)
#
#LGs_info = pd.read_csv(LGs_info_path, sep="\t", header=0)
#print(LGs_info)
#LGs_info.apply(combine_blocks, axis=1)

# Importera rensade RBX från dgn till gdb
Hämtar in punkter från DGN-filer till geodatabasen. <br>
DGN-filerna ska finnas som undermapp i projektmappen. <br>
Punkter som hämtas ska ligga i lager 20 i DGN-filerna.<br>
Om punkter ligger i annat lager kommer inga punkter att importeras. <br>
Ledningsgator och ledningar som behandlas utfrån från textfil där alla ledningar som ska köras finns med. <br>
Textfilen importeras och görs till en dataframe. <br>
I geodatabasen kommer punkterna placeras under clean_RBX_points.

In [ ]:
# Copy RBX from dgn to working_gdb feature class, one fc for each LGXXX_Y
import arcpy, os, glob
from pathlib import Path
import pandas as pd

output_fd = os.path.join(working_gdb, "clean_RBX_points")

def dgn_to_gdb(powerline): 
    LG = powerline["LG"]
    line = powerline["line"]
    littera = powerline["Littera"]
    
    os.chdir(dgn_dir)
    
    print(f"påbörjar {LG}_{line}")
    arcpy.env.overwriteOutput = True
    dgn = os.path.join(dgn_dir, LG+"_"+str(line)+".dgn") #
    print(dgn)

    output_fc = os.path.join(output_fd, LG+"_"+str(line)+"_RBX_clean") #var punkterna ska exporteras till

    #print(os.path.join(output_fd, LG+"_"+str(line)+"_RBX_clean"))
    # Create a value table that will hold the input feature classes for Merge
    vTab = arcpy.ValueTable()
    #print("vTab created")
    layername_RBX = dgn + '_Layer_LVL20'
    arcpy.MakeFeatureLayer_management(dgn + '/Point', layername_RBX, "\"Level\" = 20")
    #print("MakeFeatureLayer complete")
    vTab.addRow([layername_RBX, 2])
    #print("Row added to vTab")

    # Merge the CAD features into one feature class
    arcpy.env.workspace = output_fd
    arcpy.Merge_management(vTab, output_fc)
    print("vTab merged to output_fc")
    
    # ta bort fält som inte ska användas
    keep_fields = ["Shape", "OBJECTID"]
    all_fields = arcpy.ListFields(output_fc)
    fields_to_delete = [field.name for field in all_fields if field.name not in keep_fields]
    if fields_to_delete:
        arcpy.DeleteField_management(output_fc, fields_to_delete)

    arcpy.Delete_management(layername_RBX)
    print("Layer deleted")

    #arcpy.env.workspace = dgn_dir
 
powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
powerlines_df.apply(dgn_to_gdb, axis=1)
print("körning av cell klar")

# Near 3D - avstånd till fas för RBX
Beräknar avstånd från importerade punkter till aktuell ledning med hjälp av "Near3D_3d". <br>
Lägger till avstånd till punkterna i nya fält "AVSTAND_FAS" och "AVSTAND_HORISONTELLT" för respektive punkt. <br>
Skriptet utgår från tidigare nämnda textfil för vilka ledningsgator som ska behandlas.

In [ ]:
import arcpy, os, glob
arcpy.CheckOutExtension('3D')
from pathlib import Path
import pandas as pd

fd_RBX = os.path.join(working_gdb, 'clean_RBX_points')
arcpy.env.overwriteOutput = True

def dist2wire(fd_RBX, fc_name_RBX, fc_wires, radius):
    # Compute distance from RBX point to wire
    fc_RBX = os.path.join(fd_RBX, fc_name_RBX)
    arcpy.Near3D_3d(in_features=fc_RBX, near_features=fc_wires, search_radius=radius, location="NO_LOCATION", angle="NO_ANGLE", delta="NO_DELTA")
    print('done with Near3D '+fc_name_RBX)
    arcpy.DeleteField_management(in_table=fc_RBX, drop_field="NEAR_FID")
    arcpy.management.AlterField(in_table=fc_RBX, field="NEAR_DIST3", new_field_name="AVST_F")
    arcpy.management.AlterField(in_table=fc_RBX, field="NEAR_DIST", new_field_name="AVST_H")
    print('done with delete and alter field '+fc_name_RBX)
    
    
def RBX_3D_distance(powerline):
    LG = powerline["LG"]
    line = powerline["line"]
    littera = powerline["Littera"]
    
    print(f"{LG}_{line}, littera {littera}")
    
    fc_name_RBX = f"{LG}_{line}_RBX_clean"
    fc_name_wires = f"{LG}_{line}_fas"
    fc_wires = os.path.join(wires_gdb, fc_name_wires)
    
    if arcpy.Exists(os.path.join(fd_RBX, fc_name_RBX)):
        dist2wire(fd_RBX, fc_name_RBX, fc_wires, 6)
        print(f"Distance to wire computed for RBX {LG}_{line}")
    else:
        print(f"{os.path.join(fd_RBX, fc_name_RBX)} does not exist")

# The file pointed at by LGs_info_path contains only the powerlines that should be
# processed. So this file has to be updated each time we want to process a new set of
# powerlines. This is unpractical - consider a user prompt where the user can select
# which powerlines to process

powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
powerlines_df.apply(RBX_3D_distance, axis=1)
print("körning av cell klar")

#  Gör shp med polygoner, alla punkter, närmaste punkter
delar in punkter och polygoner i farlighetsklass
sätter minsta avstånd till fas för varje polygon. <br>

In [ ]:
# In RBX_distances below, yellow distance is set to 0.1 m higher than in the macro.
# arcpy NEAR_3D and TerraScan calculations can differ slightly, so that points
# with a distance to wire < 4.0 m according to TerraScan are further than 4.0 m
# from wire according to arcpy. This script must include also these points.
# 0.1 m is chosen because it's an easy number - the real differences won't be that big.
RBX_distances = {220: {"red": 1.6, "orange": 3, "yellow": 4.1},
                 300: {"red": 1.6, "orange": 3, "yellow": 4.1},
                 400: {"red": 2.7, "orange": 4.3, "yellow": 5.6},
                 500: {"red": 2.7, "orange": 4.3, "yellow": 5.6}}

def RBX_polys_stats(row):
    """Main function of RBX_polygons.py"""

    # Fecthing values, row by row, from the line dataframe.
    LG = row["LG"]
    line = row["line"]
    littera = row["Littera"]
    voltage = row["Spänning"]

    # Prompting which line is handled
    print(f"Doing {LG}_{line}")

    # the cleaned RBX points that where imported from dgn to gdb
    points = rbx_utils.read_points(working_gdb, LG, line, RBX_gdb_layers)
    #points = rbx_utils.read_points(working_gdb, LG, line)
    if str(type(points)) != "<class 'geopandas.geodataframe.GeoDataFrame'>":
        print("No points in list")
        return ([], [])

    # the raw RBX points from the txt file.
    # Needed because they have dZ attribute
    raw_RBX_file = os.path.join(
        powerlines_folder, LG,
        f"line_{line}", "RBX", str("RBX_raw.txt"))

    # Using "set_z_points" function to add z value to all points.
    #points = set_z_to_points(points, raw_RBX_file)
    points = rbx_utils.set_z_to_points(points, raw_RBX_file)

    # Creating polygons from distance to line.
    if len(points) > 0:
        #red_polygons, red_points = make_RBX_polygons(
        #    points, voltage, LG, littera, "red")
        #orange_polygons, orange_points = make_RBX_polygons(
        #    points, voltage, LG, littera, "orange")
        #yellow_polygons, yellow_points = make_RBX_polygons(
        #    points, voltage, LG, littera, "yellow")
        red_polygons, red_points = rbx_utils.make_RBX_polygons(
            points, voltage, LG, littera, "red", RBX_distances)
        orange_polygons, orange_points = rbx_utils.make_RBX_polygons(
            points, voltage, LG, littera, "orange", RBX_distances)
        yellow_polygons, yellow_points = rbx_utils.make_RBX_polygons(
            points, voltage, LG, littera, "yellow", RBX_distances)

    # Creating a default value and column for "KLASS_TEMP".
    points["KLASS_TEMP"] = 0
    # Using the "set_class" function for setting relevant "KLASS_TEMP" value
    # for all points.
    if len(points) > 0:
        #points = set_class(
        #    points, voltage, LG, littera, "yellow", 1)
        #points = set_class(
        #    points, voltage, LG, littera, "orange", 2)
        #points = set_class(
        #    points, voltage, LG, littera, "red", 3)
        points = rbx_utils.set_class(
            points, voltage, LG, littera, "yellow", 1, RBX_distances)
        points = rbx_utils.set_class(
            points, voltage, LG, littera, "orange", 2, RBX_distances)
        points = rbx_utils.set_class(
            points, voltage, LG, littera, "red", 3, RBX_distances)

    # Getting all the closest points from the unclipped yellow polygons.
    # The unclipped yellow polygons are used to cluster all points in groups,
    # sort them as descending by "AVSTAND_FAS" and get the point with smallest
    # distance for each group.
    yellow_polygons["polygon_ID"] = yellow_polygons.index
    #closest_points = sjoin(yellow_points, yellow_polygons, how="inner")
    closest_points = rbx_utils.sjoin(yellow_points, yellow_polygons, how="inner")
    # closest_points = closest_points.sort_values(by='AVSTAND_FAS') #### ÄNDRAT 2025-01-14
    closest_points = closest_points.sort_values(by='AVST_F')
    closest_points = closest_points.groupby('polygon_ID')

    # Sorting out the closest point for each group.
    all_closest_points_lst = []
    for spt in closest_points:
        for i in spt:
            if type(i) is not int:
                all_closest_points_lst.append(i.iloc[0])

    # Cutting the polygons.
    #_yellow_RBX = clip_polygons(yellow_polygons, orange_polygons)
    #_orange_RBX = clip_polygons(orange_polygons, red_polygons)
    _yellow_RBX = rbx_utils.clip_polygons(yellow_polygons, orange_polygons)
    _orange_RBX = rbx_utils.clip_polygons(orange_polygons, red_polygons)
    _red_RBX = red_polygons

    # Adding values to polygons from points.
    # Returning polygons and all points in groups.
    '''yellow_RBX_all = get_attributes_from_points(
        _yellow_RBX, yellow_points)
    orange_RBX_all = get_attributes_from_points(
        _orange_RBX, orange_points)
    red_RBX_all = get_attributes_from_points(_red_RBX, red_points)
    '''
    yellow_RBX_all = rbx_utils.get_attributes_from_points(
        _yellow_RBX, yellow_points)
    orange_RBX_all = rbx_utils.get_attributes_from_points(
        _orange_RBX, orange_points)
    red_RBX_all = rbx_utils.get_attributes_from_points(_red_RBX, red_points)

    # Sorting relevant data from get_attributes_from_point.
    if len(yellow_RBX_all[:-1]) > 0:
        yellow_RBX = yellow_RBX_all[:-1][0]
    else:
        yellow_RBX = []
    if len(orange_RBX_all[:-1]) > 0:
        orange_RBX = orange_RBX_all[:-1][0]
    else:
        orange_RBX = []
    if len(red_RBX_all[:-1]) > 0:
        red_RBX = red_RBX_all[:-1][0]
    else:
        red_RBX = []

    RBX_all_lines["yellow"].append(yellow_RBX)
    RBX_all_lines["orange"].append(orange_RBX)
    RBX_all_lines["red"].append(red_RBX)

    # Placing "KLASS_TEMP" and "Area" values for polygons
    all_RBX_list = []
    for cnt, gdf in enumerate([yellow_RBX, orange_RBX, red_RBX]):
        if len(gdf) > 0:
            gdf['KLASS_TEMP'] = cnt+1
            gdf['area'] = gdf.geometry.area
            all_RBX_list.append(gdf)

    # for each color, calculate RBX_area and number of clusters,
    # and append to summary.
    '''yellow_area = calculate_RBX_area(yellow_RBX)
    orange_area = calculate_RBX_area(orange_RBX)
    red_area = calculate_RBX_area(red_RBX)
    '''
    yellow_area = rbx_utils.calculate_RBX_area(yellow_RBX)
    orange_area = rbx_utils.calculate_RBX_area(orange_RBX)
    red_area = rbx_utils.calculate_RBX_area(red_RBX)

    nr_yellow = len(yellow_RBX)
    nr_orange = len(orange_RBX)
    nr_red = len(red_RBX)

    area_summary.append([LG, line, littera, yellow_area,
                        orange_area, red_area, nr_yellow, nr_orange, nr_red])

    # Creating Geodataframe out of a list
    #all_RBX = gpd.GeoDataFrame(
    all_RBX = rbx_utils.gpd.GeoDataFrame(
        pd.concat(all_RBX_list, ignore_index=True), crs="epsg:3006")

    ###################################### 2024-12-18: ORDNA FÄLT HÄR NEDANFÖR? ####################################
    all_RBX["LG"] = all_RBX["LG"]
    all_RBX.drop(
        ["area", "LG", "littera"],  # , "Shape_Length", "Shape_Area"
        axis=1,
        inplace=True)

    # Rounding up values for heights and distances.
    #all_RBX = round_values(all_RBX)
    all_RBX = rbx_utils.round_values(all_RBX)
    #all_RBX_out = sort_columns(all_RBX)
    all_RBX_out = rbx_utils.sort_columns(all_RBX)
    all_RBX_out.to_file(os.path.join(
        RBX_shape_folder, f"{LG}_{line}_RBX.shp"), crs="epsg:3006")

    # Handling all the closest points.
    # Adding KLASS_TEMP values and rounding values up.
    # Creating a dataframe and writing it to a .shp-file.
    #all_closest_points = gpd.GeoDataFrame(
    all_closest_points = rbx_utils.gpd.GeoDataFrame(
        all_closest_points_lst, crs="epsg:3006").reset_index()
    # Creaing default value and column for "KLASS_TEMP"
    all_closest_points["KLASS_TEMP"] = int(0)
    # Using the "set_class" function for setting relevant "KLASS_TEMP" value
    # for all closest points.
    if len(all_closest_points) > 0:
        '''    if len(all_closest_points) > 0:
        all_closest_points = set_class(
            all_closest_points, voltage, LG, littera, "yellow", 1)
        all_closest_points = set_class(
            all_closest_points, voltage, LG, littera, "orange", 2)
        all_closest_points = set_class(
            all_closest_points, voltage, LG, littera, "red", 3)'''
        all_closest_points = rbx_utils.set_class(
            all_closest_points, voltage, LG, littera, "yellow", 1, RBX_distances)
        all_closest_points = rbx_utils.set_class(
            all_closest_points, voltage, LG, littera, "orange", 2, RBX_distances)
        all_closest_points = rbx_utils.set_class(
            all_closest_points, voltage, LG, littera, "red", 3, RBX_distances)
    # Cleaning up GeoDataFrame from information that is not relevant.
    all_closest_points.drop(
        ["index", "index_right", "polygon_ID", "LG", "littera"],
        axis=1,
        inplace=True)
    # Rounding up values for relevant columns.
    #all_closest_points = round_values(all_closest_points)
    all_closest_points = rbx_utils.round_values(all_closest_points)
    # Sorting columns for consistency.
    #all_closest_points_out = sort_columns(all_closest_points)
    all_closest_points_out = rbx_utils.sort_columns(all_closest_points)
    all_closest_points_out.to_file(os.path.join(
        RBX_shape_folder, f"{LG}_{line}_closest_points.shp"), crs="epsg:3006")

    # Handling all the clean RBX points.
    '''points = round_values(points)
    points_out = sort_columns(points)'''
    points = rbx_utils.round_values(points)
    points_out = rbx_utils.sort_columns(points)
    # points["LG"] = points["LG"].convert_dtypes('int64')
    points_out.to_file(os.path.join(
        RBX_shape_folder, f"{LG}_{line}_all_points.shp"), crs="epsg:3006")

In [ ]:
# %% RUN
# RUN
layerlist = rbx_utils.fiona.listlayers(working_gdb)
RBX_gdb_layers = [layer for layer in rbx_utils.fiona.listlayers(
    working_gdb) if "RBX_clean" in layer]

# Dictionary for storing <color>_RBX geodataframes from all lines
RBX_all_lines = {"yellow": [], "orange": [], "red": []}
RBX_points_all_lines = {"yellow": [], "orange": [], "red": []}
area_summary = []


powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)

powerlines_df.apply(RBX_polys_stats, axis=1)

# Compute summary statistics per LG (area in square meters of yellow, orange and red RBX)
header = ['LG', 'ledning', 'littera', 'gul', 'orange',
          'röd', 'antal gula', 'antal orange', 'antal röda']
df_area_summary = pd.DataFrame(area_summary, columns=header)

# %%
print("körning av cell klar")

# Put RBX shape polygons in gdb

Skriver över geometrier från shapefiler till geodatabaser. <br>
Polygoner, all_points och closest_points hämtas från shapefiler och skrivs till geodatabasen, <br>
feature datasets
    * RBX_polygons
    * RBX_closest_points
    * RBX_all_points
Den tar alla filer i mappen, så om även de gamla ligger i samma mapp kommer dessa att köras igen, vilket tar onödig tid. Dessutom måste de då plockas bort efteråt.


In [ ]:
#Put RBX shape polygons in gdb
import arcpy
import os

# Paths to feature classes
RBX_polygons_fd = os.path.join(working_gdb,"RBX_polygons")                                                                                            # kolla även det ör som gäller vid utleverans om det är så att databasfilen ska kopieras över till utleverans
RBX_closest_points_fd = os.path.join(working_gdb,"RBX_closest_points")
RBX_all_points_fd = os.path.join(working_gdb,"RBX_all_points")

# Path to shapefile folder
#shp_folder = os.path.join(prj_dir, "RBX_polygons")


# all_points
shp_files = [os.path.join(RBX_shape_folder, f) for f in os.listdir(RBX_shape_folder) if f.endswith("_all_points.shp")]

# Loopa över listan och kopiera varje shapefil till geodatabasen
for shp_path in shp_files:
    # Skapa ett namn för featuren i geodatabasen (utan filändelse)
    feature_name = os.path.splitext(os.path.basename(shp_path))[0]
    
    # Kopiera shapefilen till geodatabasen
    arcpy.conversion.FeatureClassToFeatureClass(shp_path, RBX_all_points_fd, feature_name)
    print(f"Kopierade {shp_path} till {RBX_all_points_fd} som {feature_name}")


# closest_points
shp_files = [os.path.join(RBX_shape_folder, f) for f in os.listdir(RBX_shape_folder) if f.endswith("_closest_points.shp")]

# Loopa över listan och kopiera varje shapefil till geodatabasen
for shp_path in shp_files:
    # Skapa ett namn för featuren i geodatabasen (utan filändelse)
    feature_name = os.path.splitext(os.path.basename(shp_path))[0]
    
    # Kopiera shapefilen till geodatabasen
    arcpy.conversion.FeatureClassToFeatureClass(shp_path, RBX_closest_points_fd, feature_name)
    print(f"Kopierade {shp_path} till {RBX_closest_points_fd} som {feature_name}")
    

# polygons
shp_files = [os.path.join(RBX_shape_folder, f) for f in os.listdir(RBX_shape_folder) if f.endswith("_RBX.shp")]

# Loopa över listan och kopiera varje shapefil till geodatabasen
for shp_path in shp_files:
    # Skapa ett namn för featuren i geodatabasen (utan filändelse)
    feature_name = os.path.splitext(os.path.basename(shp_path))[0]
    
    # Kopiera shapefilen till geodatabasen
    arcpy.conversion.FeatureClassToFeatureClass(shp_path, RBX_polygons_fd, feature_name)
    print(f"Kopierade {shp_path} till {RBX_polygons_fd} som {feature_name}")
ok_to_fix_fields = True
print("körning av cell klar")

# Tar jättelång tid, borde gå att förbättra? Få fälten rätt direkt i nåt tidigare steg? Eller gör när alla featureklasser slagits ihop, om littera är fixat innan

In [ ]:
import arcpy, os, glob, shutil, datetime, time
from pathlib import Path
import pandas as pd
from datetime import date
import time

# Paths to feature datasets
RBX_polygons_fd = os.path.join(working_gdb,"RBX_polygons")       # kolla även det ör som gäller vid utleverans om det är så att databasfilen ska kopieras över till utleverans
RBX_closest_points_fd = os.path.join(working_gdb,"RBX_closest_points")
RBX_all_points_fd = os.path.join(working_gdb,"RBX_all_points")

urspr = "SWECO"
ins_met = 20
matosak_plan = 1000
matosak_hojd = 1000
lev_dat = str(date.today())


def fix_fields(powerline):
    LG = powerline["LG"]
    line = powerline["line"]
    littera = powerline["Littera"]
    LG_code = cvd_LEDNINGSGATA[LG]
    LG_date = cvd_DATE[f"{LG}_{line}"]
    print(f"Fixar fält för {LG}_{line}, skannad {LG_date}")

        
    arcpy.env.overwriteOutput = True
    RBX_polygons_fc = f"{LG}_{line}_RBX"
    RBX_closest_points_fc = f"{LG}_{line}_closest_points"
    RBX_all_points_fc = f"{LG}_{line}_all_points"
    #print(f'RBX_polygons_fc: {RBX_polygons_fc}')
    #print(f'RBX_closest_points_fc: {RBX_closest_points_fc}')
    #print(f'RBX_all_points_fc: {RBX_all_points_fc}')
    
    #print(os.path.join(RBX_polygons_fd, RBX_polygons_fc))
    if arcpy.Exists(os.path.join(RBX_polygons_fd, RBX_polygons_fc)):
        
        start_time = time.time()
        # Polygoner
        arcpy.env.workspace = RBX_polygons_fd
        arcpy.management.AddField(RBX_polygons_fc, "LITTERA", "TEXT", field_length=255, field_alias="Littera")
        arcpy.management.CalculateField(RBX_polygons_fc, "LITTERA", "".join(("'",littera,"'")))
        arcpy.management.AddField(RBX_polygons_fc, "LEDNINGSGATA", "LONG", field_alias="Ledningsgata")
        arcpy.management.CalculateField(RBX_polygons_fc, "LEDNINGSGATA", int(LG_code))
        arcpy.management.AddField(RBX_polygons_fc, "RGDTM", "DATE", field_alias="Registreringsdatum")
        arcpy.management.CalculateField(RBX_polygons_fc, "RGDTM", "".join(("'",LG_date,"'")))
        arcpy.management.AddField(RBX_polygons_fc, "Ursprung", "TEXT")
        arcpy.management.CalculateField(RBX_polygons_fc, "Ursprung", "".join(("'", urspr, "'")))
        arcpy.management.AddField(RBX_polygons_fc, "Ursprung_Datum", "DATE")
        arcpy.management.CalculateField(RBX_polygons_fc, "Ursprung_Datum", "".join(("'", lev_dat, "'")))
        arcpy.management.AddField(RBX_polygons_fc, "Insamlingsmetod", "LONG", field_alias="Inventeringsmetod")
        arcpy.management.CalculateField(RBX_polygons_fc, "Insamlingsmetod", int(ins_met))
        arcpy.management.AddField(RBX_polygons_fc, "Matosakerhet_Plan", "LONG", field_alias="Mätosäkerhet i plan")
        arcpy.management.CalculateField(RBX_polygons_fc, "Matosakerhet_Plan", int(matosak_plan))
        arcpy.management.AddField(RBX_polygons_fc, "Matosakerhet_Hojd", "LONG", field_alias="Mätosäkerhet i höjd")
        arcpy.management.CalculateField(RBX_polygons_fc, "Matosakerhet_Hojd", int(matosak_hojd))
        arcpy.management.AddField(RBX_polygons_fc, "KLASS", "SHORT", field_alias="Klass")        
        arcpy.management.CalculateField(RBX_polygons_fc, "KLASS", "!KLASS_TEMP!", "PYTHON3")
        arcpy.management.DeleteField(RBX_polygons_fc, "KLASS_TEMP")
        arcpy.management.AlterField(RBX_polygons_fc, "AVST_F", "AVSTAND_FAS", new_field_alias="Avstånd till fas [m]")
        arcpy.management.AlterField(RBX_polygons_fc, "AVST_H", "AVSTAND_HORISONTELLT", new_field_alias="Horisontellt avstånd till fas [m]")
        arcpy.management.AlterField(RBX_polygons_fc, "dz", "DELTA_HOJD", new_field_alias="Trädhöjd [m]")
        
        arcpy.management.AssignDomainToField(RBX_polygons_fc, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
        arcpy.management.AssignDomainToField(RBX_polygons_fc, "LITTERA", "cvd_LITTERA_LEDNING")
        arcpy.management.AssignDomainToField(RBX_polygons_fc, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(RBX_polygons_fc, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(RBX_polygons_fc, "Insamlingsmetod", "cvd_INMATNINGSMETOD")
        
        end_time_poly = time.time()
        elapsed_time_poly = end_time_poly - start_time
        print(f'    - polygoner fixade det tog {elapsed_time_poly:.2f} sekunder.')
        #print(f"Skriptet är klart! Det tog {elapsed_time:.2f} sekunder.")
        
        # Closest points
        arcpy.env.workspace = RBX_closest_points_fd

        arcpy.management.AddField(RBX_closest_points_fc, "LITTERA", "TEXT", field_length=255, field_alias="Littera")
        arcpy.management.CalculateField(RBX_closest_points_fc, "LITTERA", "".join(("'",littera,"'")))
        arcpy.management.AddField(RBX_closest_points_fc, "LEDNINGSGATA", "LONG", field_alias="Ledningsgata")
        arcpy.management.CalculateField(RBX_closest_points_fc, "LEDNINGSGATA", int(LG_code))
        arcpy.management.AddField(RBX_closest_points_fc, "RGDTM", "DATE", field_alias="Registreringsdatum")
        arcpy.management.CalculateField(RBX_closest_points_fc, "RGDTM", "".join(("'",LG_date,"'")))
        arcpy.management.AddField(RBX_closest_points_fc, "Ursprung", "TEXT")
        arcpy.management.CalculateField(RBX_closest_points_fc, "Ursprung", "".join(("'", urspr, "'")))
        arcpy.management.AddField(RBX_closest_points_fc, "Ursprung_Datum", "DATE")
        arcpy.management.CalculateField(RBX_closest_points_fc, "Ursprung_Datum", "".join(("'", lev_dat, "'")))
        arcpy.management.AddField(RBX_closest_points_fc, "Insamlingsmetod", "LONG", field_alias="Inventeringsmetod")
        arcpy.management.CalculateField(RBX_closest_points_fc, "Insamlingsmetod", int(ins_met))
        arcpy.management.AddField(RBX_closest_points_fc, "Matosakerhet_Plan", "LONG", field_alias="Mätosäkerhet i plan")
        arcpy.management.CalculateField(RBX_closest_points_fc, "Matosakerhet_Plan", int(matosak_plan))
        arcpy.management.AddField(RBX_closest_points_fc, "Matosakerhet_Hojd", "LONG", field_alias="Mätosäkerhet i höjd")
        arcpy.management.CalculateField(RBX_closest_points_fc, "Matosakerhet_Hojd", int(matosak_hojd))
        arcpy.management.AddField(RBX_closest_points_fc, "KLASS", "SHORT", field_alias="Klass")        
        arcpy.management.CalculateField(RBX_closest_points_fc, "KLASS", "!KLASS_TEMP!", "PYTHON3")
        arcpy.management.DeleteField(RBX_closest_points_fc, "KLASS_TEMP")
        arcpy.management.AlterField(RBX_closest_points_fc, "AVST_F", "AVSTAND_FAS", new_field_alias="Avstånd till fas [m]")
        arcpy.management.AlterField(RBX_closest_points_fc, "AVST_H", "AVSTAND_HORISONTELLT", new_field_alias="Horisontellt avstånd till fas [m]")
        arcpy.management.AlterField(RBX_closest_points_fc, "dz", "DELTA_HOJD", new_field_alias="Trädhöjd [m]")
        
        arcpy.management.AssignDomainToField(RBX_closest_points_fc, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
        arcpy.management.AssignDomainToField(RBX_closest_points_fc, "LITTERA", "cvd_LITTERA_LEDNING")
        arcpy.management.AssignDomainToField(RBX_closest_points_fc, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(RBX_closest_points_fc, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(RBX_closest_points_fc, "Insamlingsmetod", "cvd_INMATNINGSMETOD")
        
        end_time_closest_points = time.time()
        elapsed_time_closest_points = end_time_closest_points - end_time_poly
        #print("    - closest_points fixade")
        print(f'    - closest_points fixade det tog {elapsed_time_closest_points:.2f} sekunder.')

        
        # All points
        arcpy.env.workspace = RBX_all_points_fd

        arcpy.management.AddField(RBX_all_points_fc, "LITTERA", "TEXT", field_length=255, field_alias="Littera")
        arcpy.management.CalculateField(RBX_all_points_fc, "LITTERA", "".join(("'",littera,"'")))
        arcpy.management.AddField(RBX_all_points_fc, "LEDNINGSGATA", "LONG", field_alias="Ledningsgata")
        arcpy.management.CalculateField(RBX_all_points_fc, "LEDNINGSGATA", int(LG_code))
        arcpy.management.AddField(RBX_all_points_fc, "RGDTM", "DATE", field_alias="Registreringsdatum")
        arcpy.management.CalculateField(RBX_all_points_fc, "RGDTM", "".join(("'",LG_date,"'")))
        arcpy.management.AddField(RBX_all_points_fc, "Ursprung", "TEXT")
        arcpy.management.CalculateField(RBX_all_points_fc, "Ursprung", "".join(("'", urspr, "'")))
        arcpy.management.AddField(RBX_all_points_fc, "Ursprung_Datum", "DATE")
        arcpy.management.CalculateField(RBX_all_points_fc, "Ursprung_Datum", "".join(("'", lev_dat, "'")))
        arcpy.management.AddField(RBX_all_points_fc, "Insamlingsmetod", "LONG", field_alias="Inventeringsmetod")
        arcpy.management.CalculateField(RBX_all_points_fc, "Insamlingsmetod", int(ins_met))
        arcpy.management.AddField(RBX_all_points_fc, "Matosakerhet_Plan", "LONG", field_alias="Mätosäkerhet i plan")
        arcpy.management.CalculateField(RBX_all_points_fc, "Matosakerhet_Plan", int(matosak_plan))
        arcpy.management.AddField(RBX_all_points_fc, "Matosakerhet_Hojd", "LONG", field_alias="Mätosäkerhet i höjd")
        arcpy.management.CalculateField(RBX_all_points_fc, "Matosakerhet_Hojd", int(matosak_hojd))
        arcpy.management.AddField(RBX_all_points_fc, "KLASS", "SHORT", field_alias="Klass")        
        arcpy.management.CalculateField(RBX_all_points_fc, "KLASS", "!KLASS_TEMP!", "PYTHON3")
        arcpy.management.DeleteField(RBX_all_points_fc, "KLASS_TEMP")
        arcpy.management.AlterField(RBX_all_points_fc, "AVST_F", "AVSTAND_FAS", new_field_alias="Avstånd till fas [m]")
        arcpy.management.AlterField(RBX_all_points_fc, "AVST_H", "AVSTAND_HORISONTELLT", new_field_alias="Horisontellt avstånd till fas [m]")
        arcpy.management.AlterField(RBX_all_points_fc, "dz", "DELTA_HOJD", new_field_alias="Trädhöjd [m]")
        
        arcpy.management.AssignDomainToField(RBX_all_points_fc, "LEDNINGSGATA", "cvd_LEDNINGSGATA")
        arcpy.management.AssignDomainToField(RBX_all_points_fc, "LITTERA", "cvd_LITTERA_LEDNING")
        arcpy.management.AssignDomainToField(RBX_all_points_fc, "Matosakerhet_Hojd", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(RBX_all_points_fc, "Matosakerhet_Plan", "cvd_MATOSAKERHET")
        arcpy.management.AssignDomainToField(RBX_all_points_fc, "Insamlingsmetod", "cvd_INMATNINGSMETOD")
        
        end_time_all_points = time.time()
        elapsed_time_all_points = end_time_all_points - end_time_closest_points
        #print("    - all_points fixade")
        print(f'    - all_points fixade det tog {elapsed_time_all_points:.2f} sekunder.')
    
powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
df_cvd_LEDNINGSGATA = pd.read_csv(cvd_LEDNINGSGATA_path, sep="\t", header=0)
df_cvd_DATE = pd.read_csv(scandate_file, sep="\t", header=0)
cvd_LEDNINGSGATA = {df_cvd_LEDNINGSGATA.Description[i]: df_cvd_LEDNINGSGATA.Code[i] for i in range(len(df_cvd_LEDNINGSGATA))}
cvd_DATE = {f"{row['LG']}_{int(row['line'])}": row['skanningsdatum'] for _, row in df_cvd_DATE.iterrows()}
if ok_to_fix_fields == True:
    powerlines_df.apply(fix_fields, axis=1)
    ok_to_fix_fields = False
else:
    print('fix_fields not run, try running cell Put RBX shape polygons in gdb again')
print("körning av cell klar")

# Slå ihop de olika ledningarnas featureklasser till en

In [ ]:
import arcpy

results_gdb = os.path.join(local_dir, f"results_{run_ID}.gdb")

# RBX ALL POINTS
target_fc = os.path.join(results_gdb, "RBX_all_points")
# Sätt workspace till working GDB\RBX_all_points
arcpy.env.workspace = os.path.join(working_gdb, "RBX_all_points")
# lista featureklasser att kopiera
feature_classes = arcpy.ListFeatureClasses()
# Gå igenom varje featureklass och append till target GDB
for fc in feature_classes:
    arcpy.Append_management(inputs=fc, target=target_fc, schema_type="NO_TEST")

# RBX CLOSEST POINTS
target_fc = os.path.join(results_gdb, "RBX_closest_points")
# Sätt workspace till working GDB\RBX_closest_points
arcpy.env.workspace = os.path.join(working_gdb, "RBX_closest_points")
# lista featureklasser att kopiera
feature_classes = arcpy.ListFeatureClasses()
# Gå igenom varje featureklass och append till target GDB
for fc in feature_classes:
    arcpy.Append_management(inputs=fc, target=target_fc, schema_type="NO_TEST")
    
# RBX POLYGONS
target_fc = os.path.join(results_gdb, "RBX_polygons")
# Sätt workspace till working GDB\RBX_polygons
arcpy.env.workspace = os.path.join(working_gdb, "RBX_polygons")
# lista featureklasser att kopiera
feature_classes = arcpy.ListFeatureClasses()
# Gå igenom varje featureklass och append till target GDB
for fc in feature_classes:
    arcpy.Append_management(inputs=fc, target=target_fc, schema_type="NO_TEST")

print("körning av cell klar")

# Kontroll

In [ ]:
#df_cvd_DATE = pd.read_csv(scandate_file, sep="\t", header=0)
#cvd_DATE = {f"{row['LG']}_{int(row['line'])}": row['skanningsdatum'] for _, row in df_cvd_DATE.iterrows()}
#print(df_cvd_DATE)

In [ ]:
import time
import sys

# Total antal steg för progressbar
total_steps = 20

for i in range(total_steps + 1):
    # Beräkna procent
    percent = (i / total_steps) * 100
    # Skapa en progressbar
    bar = '#' * i + '-' * (total_steps - i)
    # Skriv ut progressbaren på samma rad
    print(f'\r[{bar}] {percent:.2f}%', end='', flush=True)
    time.sleep(0.1)  # Simulera arbete

print()  # För att gå till nästa rad efter att progressbaren är klar